## Step 1: Installing Required Libraries

In [1]:
# Uncomment the following line if you need to install the required libraries
!pip install gradio google-api-python-client

## Step 2: Importing Required Libraries

In [ ]:
# Section 1: Importing Libraries
# These libraries are required for Gradio UI and YouTube Data API integration
import gradio as gr
from googleapiclient.discovery import build

In [11]:
# Import dotenv package for setting environment variables 
from dotenv import load_dotenv, find_dotenv

# Import os package
import os

# Set environment variables from the .env in the local environment
load_dotenv(find_dotenv('keys1.env'))

# Retrieve API key and store as Python variable
api_key = os.getenv("YOUTUBE_API_KEY")

type(api_key)

str

## Step 3: Fetch YouTube Videos Function

In [12]:
# Section 2: Helper Functions
# Sub-section 1: Fetch YouTube Videos
def fetch_youtube_videos(subject):
    """
    Fetches the top 3 most popular YouTube videos for the given subject.
    
    Args:
        subject (str): The topic to search for.

    Returns:
        List of dictionaries containing video metadata.
    """
    # YouTube Data API Key
    YOUTUBE_API_KEY = api_key  # Replace with your valid API key
    
    # Initialize the YouTube API client
    youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)
    
    try:
        # Make the API request to fetch videos
        request = youtube.search().list(
            q=subject,              # Search query (user input)
            part="snippet",         # Fetch video metadata
            maxResults=3,           # Limit to 3 results
            order="viewCount",      # Sort by view count (popularity)
            type="video"            # Only fetch videos
        )
        response = request.execute()  # Execute the API request
        
        # Parse and store video metadata
        videos = []
        for item in response.get("items", []):
            video_data = {
                "Title": item["snippet"]["title"],
                "Channel": item["snippet"]["channelTitle"],
                "Published At": item["snippet"]["publishedAt"],
                "Description": item["snippet"]["description"],
                "Video Link": f"https://www.youtube.com/watch?v={item['id']['videoId']}"
            }
            videos.append(video_data)
        
        return videos  # Return the video metadata

    except Exception as e:
        return f"Error: {e}"  # Return error message if any

## Step 4: Gradio Application Function

In [13]:
# Sub-section 2: Gradio App Function
def app_function(subject):
    """
    Gradio app function to fetch and display video metadata.

    Args:
        subject (str): The topic to search for.

    Returns:
        str: A formatted string containing video metadata.
    """
    # Fetch video metadata using the helper function
    videos = fetch_youtube_videos(subject)
    
    # Check if an error occurred
    if isinstance(videos, str):  # If there's an error, return the error message
        return videos
    
    # Format the result as a readable string
    result = ""
    for i, video in enumerate(videos, start=1):
        result += f"### Video {i}\n"
        result += f"- **Title**: {video['Title']}\n"
        result += f"- **Channel**: {video['Channel']}\n"
        result += f"- **Published At**: {video['Published At']}\n"
        result += f"- **Description**: {video['Description']}\n"
        result += f"- **Video Link**: [Watch Here]({video['Video Link']})\n\n"
    
    return result  # Return the formatted result

## Step 5: Gradio UI Development

In [14]:
# Section 3: Gradio UI
# Define the Gradio interface
with gr.Blocks() as app:
    # Header
    gr.Markdown("# AI-Powered YouTube Content Analyzer")
    gr.Markdown("### Enter a subject to find the top 3 most popular YouTube videos.")
    
    # Input: Textbox for the subject
    user_input = gr.Textbox(label="Subject", placeholder="Enter a topic (e.g., Machine Learning)")
    
    # Output: Markdown to display the video metadata
    output = gr.Markdown(label="Video Metadata")
    
    # Button to trigger the processing
    submit_button = gr.Button("Fetch Videos")
    submit_button.click(app_function, inputs=[user_input], outputs=[output])  # Link button to the app function

## Step 6: Launching the Application

In [15]:
# Section 4: Launch the Gradio Application
# Launch the Gradio app
if __name__ == "__main__":
    app.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
